<a href="https://colab.research.google.com/github/quekhyg/NLP-Lyric-Generator/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/SMU_MITB_NLP/NLP project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1MmY0pN1b5xL_C2CijM9ImcFM-UFt4bwr/NLP project


In [ ]:
#import packages
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GRU
from keras.utils import np_utils

### Data Preprocessing

In [ ]:
path = "./songs_edit_revised/"
corpus = ''

all_files = os.listdir(path)
for file in all_files:
  with open(os.path.join(path, file)) as f:
    text = f.read()
    corpus+= text

Deep Learning Setup using Tensorflow Text Generation

In [ ]:
vocab = sorted(set(corpus))
print(f'{len(vocab)} unique characters')

62 unique characters


In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary = list(vocab),
                                              mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(corpus, 'UTF-8'))
all_ids

<tf.Tensor: shape=(56674,), dtype=int64, numpy=array([ 8, 32, 15, ..., 36, 49, 39])>

In [ ]:
ids_dataset =tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

<
V
E
R
S
E
>


W
i


In [ ]:
seq_length = 100 #100 characters
sequences = ids_dataset.batch(seq_length+1,drop_remainder=True)
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'<VERSE>\nWill you make this island\nAmazing in all ways\nSurprises every corner\nDelightful nights and da'
b'ys\nWill you take this country\nAnd turn it from a place\nTo a home that greets you\nWith smiles on every'
b' face\nWill you come on this brave journey\nWill you help to make it real\nWill you write us grand new s'
b'tories\nSongs that everyone will feel\n\n<CHORUS>\nSo will you swim the current\nWill you scale new height'
b's\nWill you make it happen\nWill you let your dreams take flight\nAnd will you make the difference\nWill '


In [ ]:
def split_input_target(sequence):
  '''
  For training, a dataset of (input,label) pairs are needed.
  At each time step, the input is the current character, and the label
  is the next character.
  '''
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("Input: ",text_from_ids(input_example).numpy())
  print("Target: ",text_from_ids(target_example).numpy())

Input:  b'<VERSE>\nWill you make this island\nAmazing in all ways\nSurprises every corner\nDelightful nights and d'
Target:  b'VERSE>\nWill you make this island\nAmazing in all ways\nSurprises every corner\nDelightful nights and da'


In [ ]:
batch_size = 128
buffer_size = 10000
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

### Model development

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 2048

In [ ]:
class BuildGRUModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru_1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru_1.get_initial_state(x)
    x, states = self.gru_1(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = BuildGRUModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    
model.summary()

(128, 100, 63) # (batch_size, sequence_length, vocab_size)
Model: "build_gru_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  32256     
                                                                 
 gru (GRU)                   multiple                  15740928  
                                                                 
 dense (Dense)               multiple                  129087    
                                                                 
Total params: 15,902,271
Trainable params: 15,902,271
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [ ]:
epochs = 100
history = model.fit(dataset,epochs=epochs)

Epoch 1/100
4/4 [==============================] - 3s 185ms/step - loss: 6.5772
Epoch 2/100
4/4 [==============================] - 1s 183ms/step - loss: 4.9843
Epoch 3/100
4/4 [==============================] - 1s 184ms/step - loss: 3.8803
Epoch 4/100
4/4 [==============================] - 1s 185ms/step - loss: 3.6900
Epoch 5/100
4/4 [==============================] - 1s 183ms/step - loss: 3.3882
Epoch 6/100
4/4 [==============================] - 1s 188ms/step - loss: 3.0633
Epoch 7/100
4/4 [==============================] - 1s 188ms/step - loss: 2.8792
Epoch 8/100
4/4 [==============================] - 1s 184ms/step - loss: 2.7194
Epoch 9/100
4/4 [==============================] - 1s 184ms/step - loss: 2.6156
Epoch 10/100
4/4 [==============================] - 1s 184ms/step - loss: 2.5101
Epoch 11/100
4/4 [==============================] - 1s 187ms/step - loss: 2.4162
Epoch 12/100
4/4 [==============================] - 1s 183ms/step - loss: 2.3457
Epoch 13/100
4/4 [===================

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model,chars_from_ids, ids_from_chars,temperature=0.5)

In [ ]:
states = None
next_char = tf.constant(['Whenever I think back'])
result = [next_char]

for n in range(200):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

Whenever I think back for me
In a heartbeat, time has passed us by
In a heartbeat, we will reach the sky
This will always be
Our people, our country, this is our family<VERSE>
Over the years, I've grown to be a part of yo 

________________________________________________________________________________


In [ ]:
verse1= result[0].numpy().decode('utf-8')
verse1

"Whenever I think back for me\nIn a heartbeat, time has passed us by\nIn a heartbeat, we will reach the sky\nThis will always be\nOur people, our country, this is our family<VERSE>\nOver the years, I've grown to be a part of yo"

In [ ]:
states = None
next_char = tf.constant(["together we"])
result2 = [next_char]

for n in range(200):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result2.append(next_char)

result2 = tf.strings.join(result2)
print(result2[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

together we can be
The change that we've been longing to see

<PRECHORUS>
You and me, we'll do our part
stand together, heart to heart
We're going to show the world where we can
We will help our fellow man
We ar 

________________________________________________________________________________


In [ ]:
prechorus = result2[0].numpy().decode('utf-8')
prechorus

"together we can be\nThe change that we've been longing to see\n\n<PRECHORUS>\nYou and me, we'll do our part\nstand together, heart to heart\nWe're going to show the world where we can\nWe will help our fellow man\nWe ar"

In [ ]:
states = None
next_char = tf.constant(["And so this I know"])
result3 = [next_char]

for n in range(200):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result3.append(next_char)

result3 = tf.strings.join(result3)
print(result3[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

And so this I know that love is there for me
Here's where I belong
We're as close as one big family
Here's where I belong
Where I keep my heart and soul
Where dreams come true for us
Where we walk together hand in hand 

________________________________________________________________________________


In [ ]:
chorus= result3[0].numpy().decode('utf-8')
chorus

"And so this I know that love is there for me\nHere's where I belong\nWe're as close as one big family\nHere's where I belong\nWhere I keep my heart and soul\nWhere dreams come true for us\nWhere we walk together hand in hand"

In [ ]:
states = None
next_char = tf.constant(["When I dream about the future"])
result4 = [next_char]

for n in range(200):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result4.append(next_char)

result4 = tf.strings.join(result4)
print(result4[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

When I dream about the future
We built it for the ones that dream
Hand in hand, we'll find new strengt
We are Singapore, 
we are Singapore
We're a nation built with love by you and me
A land to treasure right to the core
Our home 

________________________________________________________________________________


In [ ]:
verse2= result4[0].numpy().decode('utf-8')
verse2

"When I dream about the future\nWe built it for the ones that dream\nHand in hand, we'll find new strengt\nWe are Singapore, \nwe are Singapore\nWe're a nation built with love by you and me\nA land to treasure right to the core\nOur home"

In [ ]:
song_container = [verse1, prechorus,chorus,verse2,prechorus, chorus]

def to_song(song_container):
  song = ''
  for i in range(len(song_container)):
    container = []
    if i==0 or i ==3:
      song+='\n<VERSE>\n'
      container = song_container[i].split('\n')[0:5]
      song+='\n'.join(container)
    elif i==2 or i ==5:
      song+='\n<CHORUS>\n'
      container = song_container[i].split('\n')[0:4]
      song+='\n'.join(container)
    elif i ==1 or i ==4:
      song+='\n<PRECHORUS>\n'
      container = song_container[i].split('\n')[0:4]
      song+='\n'.join(container)
  
  return song

song = to_song(song_container)
song

"\n<VERSE>\nWhenever I think back for me\nIn a heartbeat, time has passed us by\nIn a heartbeat, we will reach the sky\nThis will always be\nOur people, our country, this is our family<VERSE>\n<PRECHORUS>\ntogether we can be\nThe change that we've been longing to see\n\n<PRECHORUS>\n<CHORUS>\nAnd so this I know that love is there for me\nHere's where I belong\nWe're as close as one big family\nHere's where I belong\n<VERSE>\nWhen I dream about the future\nWe built it for the ones that dream\nHand in hand, we'll find new strengt\nWe are Singapore, \nwe are Singapore\n<PRECHORUS>\ntogether we can be\nThe change that we've been longing to see\n\n<PRECHORUS>\n<CHORUS>\nAnd so this I know that love is there for me\nHere's where I belong\nWe're as close as one big family\nHere's where I belong"

In [ ]:
for line in song.split('\n'):
  print(line)


<VERSE>
Whenever I think back for me
In a heartbeat, time has passed us by
In a heartbeat, we will reach the sky
This will always be
Our people, our country, this is our family<VERSE>
<PRECHORUS>
together we can be
The change that we've been longing to see

<PRECHORUS>
<CHORUS>
And so this I know that love is there for me
Here's where I belong
We're as close as one big family
Here's where I belong
<VERSE>
When I dream about the future
We built it for the ones that dream
Hand in hand, we'll find new strengt
We are Singapore, 
we are Singapore
<PRECHORUS>
together we can be
The change that we've been longing to see

<PRECHORUS>
<CHORUS>
And so this I know that love is there for me
Here's where I belong
We're as close as one big family
Here's where I belong


In [ ]:
song_lyrics= song.split('\n')
with open("GRUfile.txt", 'w') as output:
    for row in song_lyrics:
        output.write(str(row) + '\n')

Component specific training

In [ ]:
path = "./songs_edit_revised/"
corpus = ''

all_files = os.listdir(path)
for file in all_files:
  with open(os.path.join(path, file)) as f:
    text = f.read()
    corpus+= text
#   corpus+='\t' #End of song demarcation

# corpus = corpus.lower()

In [ ]:
songs = corpus.split('\t')
components = [song.split('\n\n') for song in songs]

In [ ]:
components[0:2]

[['<verse>\nwill you make this island\namazing in all ways\nsurprises every corner\ndelightful nights and days\nwill you take this country\nand turn it from a place\nto a home that greets you\nwith smiles on every face\nwill you come on this brave journey\nwill you help to make it real\nwill you write us grand new stories\nsongs that everyone will feel',
  '<chorus>\nso will you swim the current\nwill you scale new heights\nwill you make it happen\nwill you let your dreams take flight\nand will you make the difference\nwill you seize the day\nwill you live each moment\nwill you dare to find new ways',
  '<verse>\nwill you take this city\nand turn it from a place\nto a home that greets you\nwith smiles on every face\nwill you come on this brave journey\nwill you help to make it real\nwill you write us grand new stories\nsongs that everyone will feel',
  '<chorus>\nso will you swim the current\nwill you scale new heights\nwill you make it happen\nwill you let your dreams take flight\nand

In [ ]:
def remove_token(token, bucket):
  '''
  Removes "<verse>"
  '''
  result = bucket.split('\n')
  result.pop(0)
  return result

def get_text(bucket):
  '''
  Writing all the sentences of each part into one large text string
  '''
  result = ''
  for i in range(len(bucket)):
    for j in range(len(bucket[i])):
      result+=bucket[i][j] + ' \n'
  
  return result

def get_song_part(component_list):
  verses = []
  choruses = []
  bridges = []
  others = []
  prechoruses = []

  for song in component_list:
    for part in song:
      if "<verse>" in part:
        verses.append(remove_token("<verse>",part))
      elif "<chorus>" in part:
        choruses.append(remove_token("<chorus>",part))
      elif "<bridge>" in part:
        bridges.append(remove_token("<bridge>",part))
      elif "<others>" in part:
        others.append(remove_token("<others>",part))
      else:
        prechoruses.append(remove_token("<prechorus>",part))
  
  verse = get_text(verses)
  chorus = get_text(choruses)
  bridge = get_text(bridges)
  other = get_text(others)
  prechorus = get_text(prechoruses)

  return verse, chorus, bridge, other, prechorus

In [ ]:
verse, chorus, bridge, other, prechorus = get_song_part(components)

In [ ]:
verse[0:200]

'will you make this island \namazing in all ways \nsurprises every corner \ndelightful nights and days \nwill you take this country \nand turn it from a place \nto a home that greets you \nwith smiles on ever'

In [ ]:
words =  verse.split()
unique_words = list(set(words))
print(f'Number of words: {len(words)}, Number of unique words: {len(unique_words)}')

Number of words: 4430, Number of unique words: 964


### Deep learning setup

In [ ]:
characters = sorted(list(set(verse)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

Here, X is our train array, and Y is our target array.

seq_length is the length of the sequence of characters that we want to consider before predicting a particular character.

The for loop is used to iterate over the entire length of the text and create such sequences (stored in X) and their true values (stored in Y).

In [ ]:
X = []
Y = []
length = len(verse)
seq_length = 100
for i in range(0, length-seq_length, 1):
  sequence = verse[i:i + seq_length]
  label =verse[i + seq_length]
  X.append([char_to_n[char] for char in sequence])
  Y.append(char_to_n[label])

Transforming inputs to the form of (number_of_sequences, length_of_sequence, number_of_features) for deep learning. To transform the array Y into a one-hot encoded format.

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

Experiment 1

In [ ]:
model = Sequential()
model.add(GRU(1024, input_shape=(X_modified.shape[1], X_modified.shape[2]), 
              return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(1024, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(1024))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=64)
model.save_weights('GRU_text_generator_v2.h5')

Epoch 1/100
364/364 [==============================] - 55s 129ms/step - loss: 2.7378
Epoch 2/100
364/364 [==============================] - 47s 129ms/step - loss: 2.5359
Epoch 3/100
364/364 [==============================] - 47s 129ms/step - loss: 2.3634
Epoch 4/100
364/364 [==============================] - 47s 129ms/step - loss: 2.1702
Epoch 5/100
364/364 [==============================] - 47s 130ms/step - loss: 1.9301
Epoch 6/100
364/364 [==============================] - 47s 129ms/step - loss: 1.6545
Epoch 7/100
364/364 [==============================] - 47s 129ms/step - loss: 1.3476
Epoch 8/100
364/364 [==============================] - 47s 129ms/step - loss: 1.0638
Epoch 9/100
364/364 [==============================] - 47s 129ms/step - loss: 0.8067
Epoch 10/100
364/364 [==============================] - 47s 129ms/step - loss: 0.6099
Epoch 11/100
364/364 [==============================] - 47s 129ms/step - loss: 0.4523
Epoch 12/100
364/364 [==============================] - 47s 129

In [ ]:
model.load_weights('GRU_text_generator_v2.h5')

We start off with a random row from the X array, that is an array of 100 characters. After this, we target predicting another 100 characters following X. The input is reshaped and scaled as previously and the next character with maximum probability is predicted.

seq is used to store the decoded format of the string that has been predicted till now. Next, the new string is updated, such that the first character is removed and the new predicted character is included.

In [ ]:
string_mapped = X[108] #The sequence length is only 100 so taking the wrong index of X will cause a word to be axed.
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [ ]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

' take this country \nand turn it from a place \nto a home that greets you \nwith smiles on every face \n                                                                                                                                                                                                                                                                                                                                                                                                                '

In [ ]:
verse1 = txt.split('\n')
print('Verse 1: ')
for _ in verse1: print(_)

Verse 1: 
 take this country 
and turn it from a place 
to a home that greets you 
with smiles on every face 
                                                                                                                                                                                                                                                                                                                                                                                                                


In [ ]:
string_mapped2 = X[4000] #random
full_string2 = [n_to_char[value] for value in string_mapped2]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped2,(1,len(string_mapped2), 1))
    x = x / float(len(characters))

    pred_index2 = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped2]
    full_string.append(n_to_char[pred_index2])

    string_mapped2.append(pred_index2)
    string_mapped2 = string_mapped[1:len(string_mapped2)]

In [ ]:
#combining text
txt=""
for char in full_string2:
    txt = txt+char
txt

"way \nwhat a great divine blessing \nthat our nation's still progressing \nwe're singing songs of our b"

In [ ]:
verse2 = txt.split('\n')
print('Verse 2: ')
for _ in verse2: print(_)

Verse 2: 
way 
what a great divine blessing 
that our nation's still progressing 
we're singing songs of our b


Experiment 2

In [ ]:
model = Sequential()
model.add(GRU(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=64)
model.save_weights('GRU_text_generator_100.h5')

Epoch 1/100
364/364 [==============================] - 24s 44ms/step - loss: 2.7983
Epoch 2/100
364/364 [==============================] - 16s 44ms/step - loss: 2.5842
Epoch 3/100
364/364 [==============================] - 16s 45ms/step - loss: 2.4444
Epoch 4/100
364/364 [==============================] - 17s 46ms/step - loss: 2.2782
Epoch 5/100
364/364 [==============================] - 17s 47ms/step - loss: 2.1207
Epoch 6/100
364/364 [==============================] - 18s 49ms/step - loss: 1.9541
Epoch 7/100
364/364 [==============================] - 18s 49ms/step - loss: 1.7823
Epoch 8/100
364/364 [==============================] - 18s 48ms/step - loss: 1.6232
Epoch 9/100
364/364 [==============================] - 17s 48ms/step - loss: 1.4433
Epoch 10/100
364/364 [==============================] - 17s 48ms/step - loss: 1.2870
Epoch 11/100
364/364 [==============================] - 18s 48ms/step - loss: 1.1422
Epoch 12/100
364/364 [==============================] - 18s 49ms/step - lo

In [ ]:
model.load_weights('GRU_text_generator_100.h5')

In [ ]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [ ]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

"\nwill you take this country \nand turn it from a place \nto a home that greets you \nwith smiles on every \nwe can pace areac areacan pur fand \nthere's a pac pace pur for the same \nwe ll ae wart for \nwe ll wa can to the streams \nthere s a pame \nwe lave aod fore \nwhere to the make \nwe can to the streams and free \nwe can are mane and fand \nwe ll wa can to facr \nthe tame that we can a wa whll for the streams \nwe can are camiere \nwe can the streams and wou \nthere s a pan pake and wa are aream ariln \nwhe"

In [ ]:
verse1 = txt.split('\n')
for _ in verse1: print(_)


will you take this country 
and turn it from a place 
to a home that greets you 
with smiles on every 
we can pace areac areacan pur fand 
there's a pac pace pur for the same 
we ll ae wart for 
we ll wa can to the streams 
there s a pame 
we lave aod fore 
where to the make 
we can to the streams and free 
we can are mane and fand 
we ll wa can to facr 
the tame that we can a wa whll for the streams 
we can are camiere 
we can the streams and wou 
there s a pan pake and wa are aream ariln 
whe


End of experiment